# Classification

## Imports

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import esm


In [2]:
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")

Using cache found in C:\Users\Jonathan Dragestad M/.cache\torch\hub\facebookresearch_esm_main


ImportError: cannot import name 'esmfold_structure_module_only_8M' from 'esm.pretrained' (c:\Users\Jonathan Dragestad M\.conda\envs\proteusAI\lib\site-packages\esm\pretrained.py)

## 

## Data Load

In [3]:
work_dir = os.getcwd()
data_dir = os.path.join(work_dir, '../data')

In [4]:
data = pd.read_excel(os.path.join(data_dir, 'external/antibody_info.xlsx'), header=1)
display(data)

,Antibody Name,Epitope Group,source,D614G,BA.1,BA.2,BA.2.75,BA.5,BQ.1.1,XBB,Heavy chain V gene,Heavy chain J gene,Light chain V gene,Light chain J gene,Heavy chain AA,Light chain AA
0,BD-196,C,WT convalescents,1.93,>10,>10,>10,>10,>10,>10,IGHV1-3,IGHJ6,IGKV2-58,IGKJ1,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYAMHWVRQAPGQRLE...,DVVMTQSPLSLPVTLGQPASISCRSSQSLVYSDGNTYLNWFQQRPG...
1,BD-218,A,WT convalescents,0.0104,0.00849,0.00937,0.0116,5.26,>10,>10,IGHV4-34,IGHJ1,IGKV3-11,IGKJ5,QVQLQQWGAGLLKPSETLSLTCAVYGGSFSDYFWYWIRQPPGKGLE...,EIVLTQSPATLSLSPGERATLSCRASQSVSSYLAWYQQKPGQAPRL...
2,BD-236,A,WT convalescents,0.0173,>10,>10,>10,>10,>10,>10,IGHV3-53,IGHJ6,IGKV1-9,IGKJ4,EVQLVESGGGLIQPGGSLRLSCAASGITVSSNYMSWVRQAPGKGLE...,IQLTQSPSSLSASVGDRVTITCRASQGISSYLAWYQQKPGKAPKLL...
3,BD-254,C,WT convalescents,0.25,>10,>10,>10,>10,>10,>10,IGHV3-23,IGHJ4,IGLV7-46,IGLJ2,EVQLLESGGGLVQPGGSLRLSCAASGFTLSSYAMSWVRQAPGRGLE...,QAVVTQEPSLTVSPGGTVTLTCGSSTGAVTSGHYPYWFQQKPGQAP...
4,BD-255,B,WT convalescents,0.113,>10,>10,>10,>10,>10,>10,IGHV3-30,IGHJ6,IGKV3-15,IGKJ2,QVQLVESGGGVVQPGRSLRLSCVASGFTFSNYDMHWVRQAPGKGLE...,EIVMTQSPAILSVSPGERATLSCRASQSVTRNLAWYQQKPGQAPRL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3046,BD55-6726,E2.2,SARS convalescents,>10,>10,>10,>10,>10,>10,>10,IGHV1-58,IGHJ2,IGKV3-24,IGKJ1,QMQLVQSGPEVKRPGTSVKVSCEASGFTFSSSAILWVRQPRGQRLE...,EIVLRQSPATVSLSPGERATLSCRASQSVHNYLAWFQQKPGQAPRL...
3047,BD55-6727,F2,SARS convalescents,0.293,4.37,>10,>10,>10,>10,>10,IGHV3-11,IGHJ1,IGKV1-39,IGKJ2,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMNWIRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQSISNYLNWYQQKPGKAPKL...
3048,BD55-6728,F1,SARS convalescents,>10,>10,>10,>10,>10,>10,>10,IGHV3-64,IGHJ2,IGKV1-39,IGKJ1,EVQLVESGGGLVQPGGSLRLSCAASGFTFSRFAMHWVRQAPGKGLE...,DIQMTQSPSSLSASLGDRVTITCRASQTISRYLNWYQQKPGKAPKV...
3049,BD55-6729,F1,SARS convalescents,>10,>10,>10,>10,>10,>10,>10,IGHV1-18,IGHJ6,IGLV3-25,IGLJ2,QVQLVQSGAEVKKPGTSMRVSCKASGYTFSTYGIIWVRQAPGQGLE...,SYELTQPPSVSVSPGQTARITCSGDAVATQFLYWYQQKSGQAPVMV...


In [5]:

df = data[["Antibody  Name","Heavy chain AA","Light chain AA"]]

df = pd.melt(df, id_vars="Antibody  Name", value_vars=["Heavy chain AA","Light chain AA"],var_name = "Chain ID",value_name = "chain AA")

df["identifyer"] = df["Antibody  Name"].astype(str) + " / " + df["Chain ID"]




df = df[["identifyer","chain AA"]]
# 

df = list(zip(*map(df.get, df[["identifyer","chain AA"]])))

# display(df)

[('BD-196 / Heavy chain AA',
  'QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYAMHWVRQAPGQRLEWMGWINAGNGNTRYSQKFQGRVTITRDTSASTAYMELSSLRSEDTAVYYCASRREQWLGDLGYYYYGMDVWGQGTTVTVSS'),
 ('BD-218 / Heavy chain AA',
  'QVQLQQWGAGLLKPSETLSLTCAVYGGSFSDYFWYWIRQPPGKGLEWIGEINHSGSTNYNPSLKSRVSISVDTSKNQFSLKLSSVTAADTAVYYCARGQGYGRVLLWFGEWGQGTLVTVSS'),
 ('BD-236 / Heavy chain AA',
  'EVQLVESGGGLIQPGGSLRLSCAASGITVSSNYMSWVRQAPGKGLEWVSVIYSGGSTDYADSVKGRFTISRDKSKNTLYLQMNSLRAEDTAVYYCARDLGEAGGMDVWGQGTTVTVSS'),
 ('BD-254 / Heavy chain AA',
  'EVQLLESGGGLVQPGGSLRLSCAASGFTLSSYAMSWVRQAPGRGLEWVSAVSGSGGSTYYADSVKGRFTISRDNSKNMLYLQMNSLRAEDTAIYYCALPTGASSSYSGPNYWGQGTLVTVSS'),
 ('BD-255 / Heavy chain AA',
  'QVQLVESGGGVVQPGRSLRLSCVASGFTFSNYDMHWVRQAPGKGLEWVTVISSDGNNRRYADSVKGRFTISRDNSKNMLYLQMNSLKAEDTAVYYCARDEVIAVATGEGMDVWGQGTTVTVS'),
 ('BD-319 / Heavy chain AA',
  'EVQLVESGGGLIQPGGSLRLSCAASGFTVSSNYMSWVRQAPGKGLEWVSVIYSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCARVTRAGAAGDGGAFDIWGQGTMVTVSS'),
 ('BD-320 / Heavy chain AA',
  'QVQLQESGPGLVKPSETLS

## Embed 

#### ESM 650M

In [ ]:
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

In [14]:


# # Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
# data = [
#     ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
#     ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#     ("protein2 with mask","KALTARQQEVFDLIRD<mask>ISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#     ("protein3",  "K A <mask> I S Q"),
# ]
data = df#[0:10]

batch_labels, batch_strs, batch_tokens = batch_converter(data)

batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# Extract per-residue representations (on CPU)
# only the tokens are given the model, 
# repr_layers only returns the 33 layers as representtion for every amino aids
# return_contacts predicts contracts between AA
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]

# # Generate per-sequence representations via averaging
# # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
# sequence_representations = []
# for i, tokens_len in enumerate(batch_lens):
#     sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))


# attentions scored for pos to pos
# # Look at the unsupervised self-attention map contact predictions
# import matplotlib.pyplot as plt
# for (_, seq), tokens_len, attention_contacts in zip(data, batch_lens, results["contacts"]):
#     plt.matshow(attention_contacts[: tokens_len, : tokens_len])
#     plt.title(seq)
#     plt.show()

torch.Size([129, 1280])

In [ ]:
import pickle 

with open("../data/interim/embed_EMS_650_seperate", 'wb') as f:
    pickle.dump(results["representations"][33], f)